### Download targeted ticker historical in mins

In [ ]:
import os

import pandas as pd
from tqdm import tqdm

import alpaca_trade_api as tradeapi
from alpaca_trade_api.rest import TimeFrame
import alpaca_conf_paper

In [ ]:
api = tradeapi.REST()

DATADIR = os.path.join(os.getcwd(), 'historical-market-data')

# list of symbols
SYMBOLS = ['AAPL']
FROM_DATE = '2016-01-04'
TO_DATE = '2021-12-31'

if not os.path.exists(DATADIR):
    os.mkdir(DATADIR)
    
date_range = pd.date_range(FROM_DATE, TO_DATE)

In [ ]:
for symbol in SYMBOLS:
    for fromdate in tqdm(date_range):
        
        # skip weekends
        if fromdate.dayofweek > 4:
            continue

        _from = fromdate.strftime('%Y-%m-%d')
#         _to = (fromdate + pd.Timedelta(days = 1)).strftime('%Y-%m-%d')
        _to = fromdate.strftime('%Y-%m-%d')

        file_name = f'{symbol}-{_from}.csv'
        full_name = os.path.join(DATADIR, file_name)
        
        if os.path.exists(full_name):
            continue
            
        tqdm.write(f'{_from}')
        
        # download data as a pandas dataframe format
        df = api.get_bars(
            symbol, 
            TimeFrame.Minute, 
            _from, 
            _to, 
            adjustment='raw'
        ).df


        if df.empty:
            tqdm.write(f'Error downloading data for date {_from}')
            continue

        # filter market time only
        df = df.between_time('9:30', '16:00')

        df.to_csv(full_name)
